In [ ]:
# This is for the optimization
# This cell will take the raw flat field, dark field, and ct scans and create the individual sinograms for each of the bins

import os
import numpy as np
from glob import glob
from reconstruction.ring_corr import devon_correction
from reconstruction.find_dead_pixels import find_dead_pixels
from reconstruction.inidividual_bin_recon import correct_dead_pixels
from scipy.interpolate import interp1d
from scipy.signal import medfilt

directory = r'\Material_decomposition_data/Optimization_data'
save_folder = '23_08_03_CT_inserts_all_bins'

# VARIABLES TO CHANGE
save = True

num_angles = 720  # The number of projections
scan_duration = 180  # The length of the CT scan (s)

proj_time = scan_duration / num_angles  # The scale factor (the scan time per projection/angle)

# Get all of the optimization folder names
load_folders = glob(os.path.join(directory, '23_08_03_CT_inserts_Thresholds_*'))

# The sub folder names for each of the 3 CT acquisitions for the 3 sets of tissue equivalent cylinders
sub_folders = ['test1', 'test2', 'bones']

# Iterate through the folders
for load_folder in load_folders:

    # Load the raw flat field, dark field, and water phantom scans (only the first 5 bins, discarding the first frame)
    air60 = np.load(os.path.join(load_folder, 'airscan_65s', 'Data', 'data.npy'))[1:, :, :, 0:5]
    air1 = np.sum(air60, axis=0) / (
                60 / proj_time)  # Sum all frames to get the full 60s flat field scan (scaled correctly per frame)
    dark = np.load(os.path.join(load_folder, 'darkscan_60s', 'Data', 'data.npy'))[:, :, 0:5]
    water = np.load(os.path.join(load_folder, 'water', 'Data', 'data.npy'))[1:, :, :, 0:5]
    water = np.sum(water, axis=0)  # Sum all frames

    # Get the energy thresholds from the folder name
    threshold_string = load_folder.split('\\')[-1]  # Last part of the folder path
    threshold_string = threshold_string.split('_')  # Split along the underscores
    thresholds = np.array(threshold_string[6:12], dtype='int')  # Grab the appropriate parts and cast as integers

    # Go through each sub folder
    for sub in sub_folders:

        data = np.load(os.path.join(load_folder, sub, 'Data', 'data.npy'))

        # This will cut the projections down to the correct number if there are more than necessary
        if num_angles != len(data):
            diff = abs(num_angles - len(data))
            data = data[int(np.ceil(diff / 2)):len(data) - diff // 2]

        # Process each of the individual bins, including those with bins added together
        for t1 in np.arange(5):
            for t2 in np.arange(t1 + 1, 6):
                save_file = f'{thresholds[t1]}-{thresholds[t2]}.npy'

                # Skip the file if the lower threshold is below 50 and the upper threshold greater than 90
                if not (thresholds[t1] < 50 and thresholds[t2] > 90):

                    os.makedirs(os.path.join(directory, save_folder, sub, 'Data'), exist_ok=True)
                    os.makedirs(os.path.join(directory, save_folder, sub, 'CT'), exist_ok=True)
                    os.makedirs(os.path.join(directory, save_folder, sub, 'Norm CT'), exist_ok=True)

                    # The save locations for the sinogram, CT, and normalized CT
                    sino_save_path = os.path.join(directory, save_folder, sub, 'Data', save_file)
                    ct_save_path = os.path.join(directory, save_folder, sub, 'CT', save_file)
                    norm_save_path = os.path.join(directory, save_folder, sub, 'Norm CT', save_file)

                    # If the particular bin has not been already reconstructed from another scan reconstruct it
                    if not os.path.exists(sino_save_path):

                        print(sub, save_file)

                        # Sum counts in the correct bins for all data
                        bin_data = np.sum(data[:, :, :, t1:t2], axis=-1)
                        bin_air1 = np.sum(air1[:, :, t1:t2], axis=-1)
                        bin_air60 = np.sum(air60[:, :, :, t1:t2], axis=-1)
                        bin_dark = np.sum(dark[:, :, t1:t2], axis=-1)
                        bin_water = np.sum(water[:, :, t1:t2], axis=-1)

                        # Create the dead pixel mask
                        dpm = find_dead_pixels(np.sum(bin_air60[0:6], axis=0), np.sum(bin_air60[6:], axis=0), bin_dark)

                        # Correct raw data and air data for dead pixels
                        print('Correcting dead pixels in raw data')
                        bin_data = correct_dead_pixels(bin_data, dpm, True, False)
                        print('Correcting dead pixels in air scan')
                        bin_air1 = correct_dead_pixels(bin_air1, dpm, True, True)

                        # Do the -ln(I/I0) correction
                        sino = np.squeeze(np.log(bin_air1 + 0.01) - np.log(bin_data + 0.01))

                        bin_air1 = np.squeeze(bin_air1)

                        # Correct for pixel non-uniformities (ring artifacts)
                        corr_array = devon_correction(bin_water, bin_air1 * (60 / proj_time), dpm, num_bins=1)

                        # Correct the sinogram
                        sino = np.multiply(corr_array, sino)

                        # Additional correction
                        sum_corr = np.sum(sino, axis=0)
                        sum_dpm = np.ones((24, 576))
                        xpts = np.arange(576)
                        for z in range(24):
                            f = interp1d(xpts, medfilt(sum_corr[z], 21))

                            diff = np.abs(f(xpts) - sum_corr[z])

                            sum_dpm[z][diff > 10] = np.nan

                        sino = np.squeeze(correct_dead_pixels(sino, sum_dpm, True, False))

                        # Save the sinogram and display it
                        if save:
                            np.save(sino_save_path, sino)

In [ ]:
## This cell will create all the CT scans from each of the sinograms

import os
import numpy as np
from glob import glob
from reconstruction.inidividual_bin_recon import reconstruct_CT

directory = r'\Material_decomposition_data/Optimization_data'
save_folder = '23_08_03_CT_inserts_all_bins'

# VARIABLES TO CHANGE
save = True

num_angles = 720  # The number of projections
scan_duration = 180  # The length of the CT scan (s)
offset = -1.33
filter_type = 'Al'  # Al or Cu most likely
filter_thickness = 6

# The sub folder names for each of the 3 CT acquisitions for the 3 sets of tissue equivalent cylinders
sub_folders = ['test1', 'test2', 'bones']

# Go through each sub folder
for sub in sub_folders:

    sino_files = glob(os.path.join(directory, save_folder, sub, 'Data', '*'))

    for file in sino_files:

        # Set the path to load the sinogram
        load_path = file

        # Set the file and only the file's name
        file = file.split('\\')[-1]
        print(sub, file)

        os.makedirs(os.path.join(directory, save_folder, sub, 'CT'), exist_ok=True)

        # The save locations for the CT
        ct_save_path = os.path.join(directory, save_folder, sub, 'CT', f'CT_{file}')

        # Load the sinogram
        sino = np.load(load_path)

        # Reconstruct the sinogram
        ct = reconstruct_CT(sino, h_offset=offset)

        # Save the CT scan
        if save:
            np.save(ct_save_path, ct)

In [ ]:
## This cell will create normalize the CT scans to HU

import os
import numpy as np
from reconstruction.inidividual_bin_recon import normalize_ct
from glob import glob

directory = r'\Material_decomposition_data/Optimization_data'
save_folder = '23_08_03_CT_inserts_all_bins'

# VARIABLES TO CHANGE
save = True

# The sub folder names for each of the 3 CT acquisitions for the 3 sets of tissue equivalent cylinders
sub_folders = ['test1', 'test2', 'bones']

# Go through each sub folder
for sub in sub_folders:

    ct_files = glob(os.path.join(directory, save_folder, sub, 'CT', '*'))

    water_mask = np.load(os.path.join(directory, save_folder, sub, 'water_mask.npy'))

    for file in ct_files:

        # Set the path to load the CT scan
        load_path = file

        # Set the file and only the file's name
        file = file.split('\\')[-1]
        print(sub, file)

        os.makedirs(os.path.join(directory, save_folder, sub, 'Norm CT'), exist_ok=True)

        # The save locations for the CT
        norm_save_path = os.path.join(directory, save_folder, sub, 'Norm CT', f'Norm_{file}')

        # Load the CT scan
        ct = np.load(load_path)

        # Load the good slice indices for the particular CT scan
        gs = np.load(os.path.join(directory, save_folder, sub, 'Slice info', file))
        norm_slice = gs[len(gs) // 2]  # The slice to calculate the normalization values from

        ct = normalize_ct(ct, water_mask, water_slice=norm_slice)

        # Save the CT scan
        if save:
            np.save(norm_save_path, ct)

In [1]:
# This will take all of the normalized data and the tissue cylinder masks and optimize based on the ideal energy bin
# It will record the RMSPE for each cylinder (effective atomic number and relative electron density) as well as the total RMSPE for the whole set
# The model will be calibrated on half of the slices from the CT scan and evaluated on the other half, the slices used for both will be randomized.

import os
import numpy as np

from decomposition.bourque import solve_for_c, solve_for_b
from glob import glob
from natsort import natural_keys

from datetime import datetime

directory = r'\Material_decomposition_data/Optimization_data'
save_folder = '23_08_03_CT_inserts_all_bins'

# The sub folder names for each of the 3 CT acquisitions for the 3 sets of tissue equivalent cylinders
sub_folders = ['test1', 'test2', 'bones']

# The labels for each of the vials in the three subsets
sub_labels = {'test1': ['Water', 'BRN-SR2 Brain', 'SB3 Cortical Bone', 'LN-450 Lung', 'AP6 Adipose', 'Solid Water'],
              'test2': ['Water', r'CB2-50%', 'LV1 Liver', 'LN-300 Lung', 'BR-12 Breast'],
              'bones': ['Water', r'CB2-30%', 'B-200 Bone', 'IB Inner Bone']
              }

# Ideal values 120 kVp full spectrum and BC electron sheet densities
z_values = {
    'Water': 7.693,
    'LN-300 Lung': 7.796,
    'LN-450 Lung': 7.719,
    'BR-12 Breast': 7.011,
    'AP6 Adipose': 6.358,
    'Solid Water': 7.778,
    'LV1 Liver': 7.774,
    'BRN-SR2 Brain': 6.358,
    'IB Inner Bone': 10.666,
    'B-200 Bone': 10.514,
    'CB2-30%': 11.097,
    'CB2-50%': 12.506,
    'SB3 Cortical Bone': 13.570
}

rho_values = {
    'Water': 1,
    'LN-300 Lung': 0.28,
    'LN-450 Lung': 0.465,
    'BR-12 Breast': 0.957,
    'AP6 Adipose': 0.937,
    'Solid Water': 0.987,
    'LV1 Liver': 1.077,
    'BRN-SR2 Brain': 1.049,
    'IB Inner Bone': 1.106,
    'B-200 Bone': 1.105,
    'CB2-30%': 1.274,
    'CB2-50%': 1.467,
    'SB3 Cortical Bone': 1.691
}

ct_files = glob(os.path.join(directory, save_folder, 'test1', 'Norm CT', '*'))
filenames = []

# Obtain only the filenames of the CT files, not the whole path
for ctf in ct_files:
    filenames.append(ctf.split('\\')[-1])

# Sort filenames so that the lower bin thresholds appear first
filenames.sort(key=natural_keys)
num_files = len(filenames)

# Go through each of the filenames (energy threshold and pair it with a higher threshold)(Don't double up)
for lowidx in np.arange(0, num_files-1):
    file_low = filenames[lowidx]
    low_thresholds = file_low.split('_')[-1]
    low_thresholds = low_thresholds.split('-')
    low_t1 = int(low_thresholds[0])
    low_t2 = int(low_thresholds[1][0:-4])

    for highidx in np.arange(lowidx+1, num_files):
        file_high = filenames[highidx]
        high_thresholds = file_high.split('_')[-1]
        high_thresholds = high_thresholds.split('-')
        high_t1 = int(high_thresholds[0])
        high_t2 = int(high_thresholds[1][0:-4])

        # Proceed if the bins don't overlap and the higher bin is a higher energy
        if not low_t1 == high_t1 and high_t1 > low_t2 + 5:
            print(f'{low_t1}-{low_t2}', f'{high_t1}-{high_t2}')
            start = datetime.now().timestamp()
            # Go through each of the calibration sub folders and get the calibration values
            calib_z = []
            calib_rho = []
            low_hu_calib = []
            high_hu_calib = []

            for idx, calib_sub in enumerate(sub_folders):

                calib_labels = sub_labels[calib_sub]

                # Skip the water values if we're past the first sub folder
                if idx > 0:
                    calib_labels = calib_labels[1:]
                sub_calib_z = []
                sub_calib_rho = []
                for label in calib_labels:
                    calib_z.append(z_values[label])
                    calib_rho.append(rho_values[label])

                # Load the calibration data
                low_data = np.load(os.path.join(directory, save_folder, calib_sub, 'Norm CT', file_low))
                high_data = np.load(os.path.join(directory, save_folder, calib_sub, 'Norm CT', file_high))

                # Load the masks for the cylinders
                masks = np.load(os.path.join(directory, save_folder, calib_sub, 'mat_decomp_masks.npy'))

                # Load the appropriate calibration slices
                slice_folder = os.path.join(directory, save_folder, calib_sub, 'Slice info')
                slice_path = os.path.join(slice_folder, f'{low_t1}-{low_t2}_{high_t1}-{high_t2}.npy')

                if os.path.exists(slice_path):
                    good_slices = np.load(slice_path)
                else:
                    low_slices = np.load(os.path.join(slice_folder, f'CT_{low_t1}-{low_t2}.npy'))
                    high_slices = np.load(os.path.join(slice_folder, f'CT_{high_t1}-{high_t2}.npy'))

                    good_slices = np.intersect1d(low_slices, high_slices)
                    np.random.shuffle(good_slices)

                    np.save(slice_path, good_slices)

                calib_slices = good_slices[0:len(good_slices)//2]
                num_slices = len(calib_slices)


                # Skip the water values if we're past the first sub folder
                if idx > 0:
                    masks = masks[1:]

                for i, mask in enumerate(masks):

                    temp_low_hu = np.zeros(num_slices)
                    temp_high_hu = np.zeros(num_slices)

                    for cidx, cz in enumerate(calib_slices):
                        temp_low_hu[cidx] = np.nanmean(low_data[cz] * mask)
                        temp_high_hu[cidx] = np.nanmean(high_data[cz] * mask)

                    low_hu_calib.append(np.mean(temp_low_hu))
                    high_hu_calib.append(np.mean(temp_high_hu))

            low_hu_calib = np.array(low_hu_calib)
            high_hu_calib = np.array(high_hu_calib)

            calib_path = os.path.join(directory, save_folder, f'Calibrated_values')
            os.makedirs(calib_path, exist_ok=True)

            for K in np.arange(2, 8):
                # Solve for c
                K_file = f'C_{low_t1}-{low_t2}_{high_t1}-{high_t2}_K{K}.npy'
                # if not os.path.exists(os.path.join(calib_path, K_file)):
                c = solve_for_c(calib_z, low_hu_calib, high_hu_calib, K=K)
                if not c is None:
                    np.save(os.path.join(calib_path, K_file), np.array([c]))

            for M in np.arange(2, 8):
                # Solve for b_low and b_high
                M_file = f'B_{low_t1}-{low_t2}_{high_t1}-{high_t2}_M{M}.npy'
                # if not os.path.exists(os.path.join(calib_path, M_file)):
                b_low = solve_for_b(calib_z, calib_rho, low_hu_calib, M=M)
                b_high = solve_for_b(calib_z, calib_rho, high_hu_calib, M=M)
                if not b_low is None or b_high is None:
                    np.save(os.path.join(calib_path, M_file), np.array([b_low, b_high]))
            print(f'{datetime.now().timestamp() - start:0.3f} s')
            print()

35-45 55-65


C:\Users\drich\PycharmProjects\ReconRevamp\material_decomp\bourque_method\bourque.py:37: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c, c_res, c_rank, c_s = np.linalg.lstsq(L, z_calib)
C:\Users\drich\PycharmProjects\ReconRevamp\material_decomp\bourque_method\bourque.py:133: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  b, b_res, b_rank, b_s = np.linalg.lstsq(F, u)


1.329 s

35-45 55-70
0.779 s

35-45 55-75
0.892 s

35-45 55-80
0.713 s

35-45 55-85
0.813 s

35-45 55-90
0.707 s

35-45 55-95
0.760 s

35-45 55-100
0.724 s

35-45 55-110
0.741 s

35-45 55-115
0.739 s

35-45 55-120
0.756 s

35-45 60-70
1.031 s

35-45 60-75
0.848 s

35-45 60-80
0.790 s

35-45 60-85
0.800 s

35-45 60-90
0.740 s

35-45 60-95
0.725 s

35-45 60-100
0.785 s

35-45 60-105
0.792 s

35-45 60-110
0.796 s

35-45 60-120
0.770 s

35-45 65-75
0.750 s

35-45 65-80
0.697 s

35-45 65-85
0.710 s

35-45 65-95
0.714 s

35-45 65-100
0.680 s

35-45 65-105
0.749 s

35-45 65-110
0.738 s

35-45 65-115
0.765 s

35-45 65-120
0.718 s

35-45 70-80
0.673 s

35-45 70-85
0.679 s

35-45 70-90
0.723 s

35-45 70-95
0.873 s

35-45 70-100
0.662 s

35-45 70-105
0.756 s

35-45 70-110
0.741 s

35-45 70-115
0.764 s

35-45 70-120
0.762 s

35-45 75-85
0.766 s

35-45 75-90
0.669 s

35-45 75-95
0.699 s

35-45 75-100
0.739 s

35-45 75-105
0.701 s

35-45 75-110
0.778 s

35-45 75-115
0.689 s

35-45 75-120
0.744 s

35

c:\users\drich\anaconda3\envs\pcd_ct_recon\lib\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\drich\anaconda3\envs\pcd_ct_recon\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


c did not converge
c did not converge
c did not converge
c did not converge
c did not converge
c did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
0.611 s

35-45 85-95
0.775 s

35-45 85-100
0.698 s

35-45 85-105
0.710 s

35-45 85-110
0.670 s

35-45 85-115
0.730 s

35-45 85-120
0.661 s

35-45 90-100
0.441 s

35-45 90-105
0.223 s

35-45 90-110
0.320 s

35-45 90-115
0.203 s

35-45 90-120
0.599 s

35-45 95-105
0.659 s

35-45 95-110
0.681 s

35-45 95-115
0.714 s

35-45 95-120
0.687 s

35-45 100-110
0.754 s

35-45 100-120
c did not converge
c did not converge
c did not converge
c did not converge
c did not converge
c did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converge
b did not converg

In [3]:
# This will take all of the normalized data and the tissue cylinder masks and optimize based on the ideal energy bin
# It will record the RMSPE for each cylinder (effective atomic number and relative electron density) as well as the total RMSPE for the whole set
# The model will be calibrated on half of the slices from the CT scan and evaluated on the other half, the slices used for both will be randomized.

# This will evaluate calculate the effective Z and rho images

import os
import numpy as np
import pandas as pd
from decomposition.bourque import solve_for_z, solve_for_rho
from glob import glob

from datetime import datetime

directory = r'\Material_decomposition_data/Optimization_data'
save_folder = '23_08_03_CT_inserts_all_bins'

# The sub folder names for each of the 3 CT acquisitions for the 3 sets of tissue equivalent cylinders
sub_folders = ['test1', 'test2', 'bones']

# The labels for each of the vials in the three subsets
sub_labels = {'test1': ['Water', 'BRN-SR2 Brain', 'SB3 Cortical Bone', 'LN-450 Lung', 'AP6 Adipose', 'Solid Water'],
              'test2': ['Water', r'CB2-50%', 'LV1 Liver', 'LN-300 Lung', 'BR-12 Breast'],
              'bones': ['Water', r'CB2-30%', 'B-200 Bone', 'IB Inner Bone']
              }

# Ideal values 120 kVp full spectrum and BC electron sheet densities
z_values = {
    'Water': 7.693,
    'LN-300 Lung': 7.796,
    'LN-450 Lung': 7.719,
    'BR-12 Breast': 7.011,
    'AP6 Adipose': 6.358,
    'Solid Water': 7.778,
    'LV1 Liver': 7.774,
    'BRN-SR2 Brain': 6.358,
    'IB Inner Bone': 10.666,
    'B-200 Bone': 10.514,
    'CB2-30%': 11.097,
    'CB2-50%': 12.506,
    'SB3 Cortical Bone': 13.570
}

rho_values = {
    'Water': 1,
    'LN-300 Lung': 0.28,
    'LN-450 Lung': 0.465,
    'BR-12 Breast': 0.957,
    'AP6 Adipose': 0.937,
    'Solid Water': 0.987,
    'LV1 Liver': 1.077,
    'BRN-SR2 Brain': 1.049,
    'IB Inner Bone': 1.106,
    'B-200 Bone': 1.105,
    'CB2-30%': 1.274,
    'CB2-50%': 1.467,
    'SB3 Cortical Bone': 1.691
}

num_materials = len(list(z_values.keys()))

xx, yy, = np.mgrid[:512, :512]
circle = (xx - 512 // 2) ** 2 + (yy - 512 // 2) ** 2

calib_path = os.path.join(directory, save_folder, f'Calibrated_values')
filenames = glob(os.path.join(directory, save_folder, 'test1', 'Slice info', '[!CT_]*'))
print(len(filenames))

eval_columns = ['Low t1' , 'Low t2', 'High t1', 'High t2', 'K', 'M', 'Z RMSE', 'rho RMSE', 'Z noise', 'rho noise', 'Lung Z RMSE', 'Lung rho RMSE',
                'Lung Z noise', 'Lung rho noise']

num_combinations = len(filenames) * 36

eval_results = np.zeros((num_combinations, len(eval_columns)))

eval_index = 0

# Go through each of the filenames
for combo_file in filenames:

    file = combo_file.split('\\')[-1]

    low_thresholds = file.split('_')[0]
    high_thresholds = file.split('_')[1][:-4]
    print(low_thresholds, high_thresholds)

    low_t1, low_t2 = low_thresholds.split('-')
    high_t1, high_t2 = high_thresholds.split('-')

    low_file = f'Norm_CT_{low_t1}-{low_t2}.npy'
    high_file = f'Norm_CT_{high_t1}-{high_t2}.npy'

    threshold_file_name = f'{low_t1}-{low_t2}_{high_t1}-{high_t2}'

    start = datetime.now().timestamp()

    mask_example = np.load(os.path.join(directory, save_folder, 'test1', 'mat_decomp_masks.npy'))[0]
    num_mask_pixels = len(mask_example[np.logical_not(np.isnan(mask_example))])

    # Now we'll go through the K and M values for c and b and calculate the results of the decomposition for the pixels
    for K in np.arange(2, 8):

        if os.path.exists(os.path.join(calib_path, f'C_{threshold_file_name}_K{K}.npy')):
            c = np.load(os.path.join(calib_path, f'C_{threshold_file_name}_K{K}.npy'))[0]

            for M in np.arange(2, 8):
                # print(K, M)
                if os.path.exists(os.path.join(calib_path, f'B_{threshold_file_name}_M{M}.npy')):
                    b_low, b_high = np.load(os.path.join(calib_path, f'B_{threshold_file_name}_M{M}.npy'))

                    z_dict = {}
                    rho_dict = {}
                    for sidx, sub in enumerate(sub_folders):

                        low_data = np.load(os.path.join(directory, save_folder, sub, 'Norm CT', low_file))
                        high_data = np.load(os.path.join(directory, save_folder, sub, 'Norm CT', high_file))

                        good_slices = np.load(os.path.join(directory, save_folder, sub, 'Slice info', f'{threshold_file_name}.npy'))
                        good_slices = good_slices[len(good_slices)//2:]
                        num_slices = len(good_slices)

                        masks = np.load(os.path.join(directory, save_folder, sub, 'mat_decomp_masks.npy'))

                        for midx, mask in enumerate(masks):

                            mat_type = sub_labels[sub][midx]

                            low_mask_array = np.zeros((num_slices, num_mask_pixels))
                            high_mask_array = np.zeros((num_slices, num_mask_pixels))

                            for zidx, z in enumerate(good_slices):

                                # Get only the values within the specified cylinder
                                temp_low = low_data[z] * mask
                                temp_high = high_data[z] * mask

                                # Remove nan values
                                temp_low = temp_low[np.logical_not(np.isnan(temp_low))]
                                temp_high = temp_high[np.logical_not(np.isnan(temp_high))]

                                low_mask_array[zidx] = temp_low
                                high_mask_array[zidx] = temp_high

                            low_mask_array = low_mask_array.flatten()
                            high_mask_array = high_mask_array.flatten()

                            z_calc = solve_for_z(low_mask_array, high_mask_array, c)
                            rho_calc = solve_for_rho(z_calc, low_mask_array, high_mask_array, b_low, b_high)

                            if mat_type == 'Water':
                                if 'Water' in z_dict:
                                    z_dict[mat_type] = np.concatenate((z_dict[mat_type], z_calc))
                                    rho_dict[mat_type] = np.concatenate((rho_dict[mat_type], rho_calc))
                                else:
                                    z_dict[mat_type] = z_calc.flatten()
                                    rho_dict[mat_type] = rho_calc.flatten()
                            else:
                                z_dict[mat_type] = z_calc.flatten()
                                rho_dict[mat_type] = rho_calc.flatten()

                    # RMSE values
                    mean_rmse_z = []
                    mean_rmse_rho = []
                    noise_z = []
                    noise_rho = []
                    lung_rmse_z = 0
                    lung_rmse_rho = 0
                    lung_noise_z = 0
                    lung_noise_rho = 0
                    for ik, key in enumerate(list(z_dict.keys())):
                        z_real = z_values[key]
                        rho_real = rho_values[key]

                        z_key = z_dict[key]
                        rho_key = rho_dict[key]

                        z_rmse = np.reshape(z_key, (len(z_key)//num_mask_pixels, num_mask_pixels))
                        rho_rmse = np.reshape(rho_key, (len(rho_key)//num_mask_pixels, num_mask_pixels))

                        z_rmse = np.mean(z_rmse, axis=1)
                        rho_rmse = np.mean(rho_rmse, axis=1)

                        z_rmse = np.sqrt(np.mean(np.square(((z_real - z_rmse) / z_real)))) * 100
                        rho_rmse = np.sqrt(np.mean(np.square(((rho_real - rho_rmse) / rho_real)))) * 100

                        z_mean = np.mean(z_key)
                        rho_mean = np.mean(rho_key)

                        z_std = np.std(z_key) / z_mean
                        rho_std = np.std(rho_key) / rho_mean

                        if key == 'LN-300 Lung':
                            lung_rmse_z = z_rmse
                            lung_rmse_rho = rho_rmse
                            lung_noise_z = z_std
                            lung_noise_rho = rho_std
                        else:
                            mean_rmse_z.append(z_rmse)
                            mean_rmse_rho.append(rho_rmse)
                            noise_z.append(z_std)
                            noise_rho.append(rho_std)

                    if np.any(np.array(noise_z) < 0) or np.any(np.array(noise_rho) < 0):
                        print(f'Unreal values found with K = {K} and M = {M}, disregard.')
                    else:

                        eval_results[eval_index] = [int(low_t1), int(low_t2), int(high_t1), int(high_t2), K, M, np.mean(mean_rmse_z),
                                                    np.mean(mean_rmse_rho), np.mean(noise_z), np.mean(noise_rho), lung_rmse_z, lung_rmse_rho,
                                                    lung_noise_z, lung_noise_rho]

                        eval_index += 1

    print(f'{datetime.now().timestamp() - start:0.3f}')

eval_results = eval_results[0:eval_index]  # Remove last rows of zeros if some data was found to be invalid.
eval_results = pd.DataFrame(eval_results, columns=eval_columns)
eval_results.to_csv(os.path.join(directory, save_folder, f'optimization_results.csv'))

1291
35-100 110-120
Unreal values found with K = 6 and M = 2, disregard.
Unreal values found with K = 6 and M = 3, disregard.
Unreal values found with K = 6 and M = 4, disregard.
Unreal values found with K = 6 and M = 5, disregard.
Unreal values found with K = 6 and M = 6, disregard.
Unreal values found with K = 6 and M = 7, disregard.
Unreal values found with K = 7 and M = 2, disregard.
Unreal values found with K = 7 and M = 3, disregard.
Unreal values found with K = 7 and M = 4, disregard.
Unreal values found with K = 7 and M = 5, disregard.
Unreal values found with K = 7 and M = 6, disregard.
Unreal values found with K = 7 and M = 7, disregard.
21.355
35-45 100-110
Unreal values found with K = 7 and M = 2, disregard.
Unreal values found with K = 7 and M = 3, disregard.
Unreal values found with K = 7 and M = 4, disregard.
Unreal values found with K = 7 and M = 5, disregard.
Unreal values found with K = 7 and M = 6, disregard.
Unreal values found with K = 7 and M = 7, disregard.
14.88

In [8]:
# This will look through the optimization results and find the best parameters

import os
import numpy as np
import pandas as pd
from scipy.optimize import minimize, LinearConstraint

directory = r'\Material_decomposition_data/Optimization_data'
save_folder = '23_08_03_CT_inserts_all_bins'

eval_results = pd.read_csv(os.path.join(directory, save_folder, f'optimization_results.csv'))
eval_results = eval_results[eval_results['Lung Z noise'] > 0]
eval_results = eval_results[eval_results['Lung rho noise'] > 0]

# Initial guess for the weights
initial_weights = [0.4, 0.35, 0.1, 0.15]

norm_results = eval_results

z_rmse = norm_results['Z RMSE'].values * initial_weights[0]
rho_rmse = norm_results['rho RMSE'].values * initial_weights[1]

lung_z_rmse = norm_results['Lung Z RMSE'].values * initial_weights[2]
lung_rho_rmse = norm_results['Lung rho RMSE'].values * initial_weights[3]


full_results = z_rmse + rho_rmse + lung_z_rmse + lung_rho_rmse

# full_results = eval_results['Z RMSE'].values + eval_results['rho RMSE'].values

idx = np.argmin(full_results)

print(f'Initial weights: {initial_weights}')
print(idx)
# print(f'{np.min(full_results):0.9f}')
print(eval_results.iloc[idx])
print()

# Define the bounds for the variations of each weight
weight_variation_bounds = [(w - 0.7 * w, w + 0.7 * w) for w in initial_weights]

# Define a constraint that ensures the weights sum to 1
sum_constraint = LinearConstraint(np.ones_like(initial_weights), [1], [1])

# Define the objective function to minimize
def objective_function(weights):
    z_rmse = norm_results['Z RMSE'].values * weights[0]
    rho_rmse = norm_results['rho RMSE'].values * weights[1]
    lung_z_rmse = norm_results['Lung Z RMSE'].values * weights[2]
    lung_rho_rmse = norm_results['Lung rho RMSE'].values * weights[3]
    full_results = z_rmse + rho_rmse + lung_z_rmse + lung_rho_rmse
    return np.min(full_results)

# Perform constrained optimization
result = minimize(objective_function, initial_weights, method='trust-constr', constraints=[sum_constraint], bounds=weight_variation_bounds)

# Extract the optimized weights
optimized_weights = result.x

# Print the results
print(f'Initial weights: {initial_weights}')
print(f'Optimized weights: {optimized_weights}')
print(f'Optimized objective value: {result.fun}')

z_rmse = norm_results['Z RMSE'].values * initial_weights[0]
rho_rmse = norm_results['rho RMSE'].values * initial_weights[1]

lung_z_rmse = norm_results['Lung Z RMSE'].values * initial_weights[2]
lung_rho_rmse = norm_results['Lung rho RMSE'].values * initial_weights[3]


full_results = z_rmse + rho_rmse + lung_z_rmse + lung_rho_rmse
print(idx)
print(eval_results.iloc[idx])

Initial weights: [0.4, 0.35, 0.1, 0.15]
10056
Unnamed: 0        11333.000000
Low t1               40.000000
Low t2               50.000000
High t1              65.000000
High t2              95.000000
K                     5.000000
M                     5.000000
Z RMSE                0.700633
rho RMSE              0.560133
Z noise               0.074596
rho noise             0.039443
Lung Z RMSE           0.633527
Lung rho RMSE         0.581666
Lung Z noise          0.316447
Lung rho noise        1.143853
Name: 11333, dtype: float64

Initial weights: [0.4, 0.35, 0.1, 0.15]
Optimized weights: [0.1201599  0.59488084 0.03000782 0.25495144]
Optimized objective value: 0.5621637674646869
10056
Unnamed: 0        11333.000000
Low t1               40.000000
Low t2               50.000000
High t1              65.000000
High t2              95.000000
K                     5.000000
M                     5.000000
Z RMSE                0.700633
rho RMSE              0.560133
Z noise               0.

c:\users\drich\anaconda3\envs\pcd_ct_recon\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:186: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)
